In [30]:
# EDA 사전 세팅
%matplotlib inline

import time
import os
import openai

os.environ['KMP_WARNINGS'] = 'off'
from functools import reduce
import matplotlib as mpl
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic", 

        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

In [31]:
data = pd.read_csv('/Users/choejeehyuk/DNA/🚲UOS 빅데이터 알고리즘 경진대회/train.csv')
sample_submission_raw = pd.read_csv('/Users/choejeehyuk/DNA/🚲UOS 빅데이터 알고리즘 경진대회/sample_submission.csv')

In [32]:
data['일시'] = data['일시'].astype('str')

#data['MS'] = pd.to_datetime(data['일시']).dt.to_period('m')
data['일시'] = pd.to_datetime(data['일시']).dt.date
data = data.set_index('일시')
data.sort_index(inplace=True)
data
    

,광진구,동대문구,성동구,중랑구
일시,,,,
2018-01-01,0.592,0.368,0.580,0.162
2018-01-02,0.840,0.614,1.034,0.260
2018-01-03,0.828,0.576,0.952,0.288
2018-01-04,0.792,0.542,0.914,0.292
2018-01-05,0.818,0.602,0.994,0.308
...,...,...,...,...
2021-12-27,3.830,3.416,2.908,2.350
2021-12-28,4.510,3.890,3.714,2.700
2021-12-29,4.490,3.524,3.660,2.524


In [33]:
data.isnull().sum()

광진구     0
동대문구    0
성동구     0
중랑구     0
dtype: int64

In [34]:
print(data[data['광진구'] <= 0])
print(data[data['성동구'] <= 0])
print(data[data['동대문구'] <= 0])
print(data[data['중랑구'] <= 0])

            광진구  동대문구  성동구  중랑구
일시                             
2019-09-07  0.0   0.0  0.0  0.0
            광진구  동대문구  성동구  중랑구
일시                             
2019-09-07  0.0   0.0  0.0  0.0
            광진구  동대문구  성동구  중랑구
일시                             
2019-09-07  0.0   0.0  0.0  0.0
            광진구  동대문구  성동구  중랑구
일시                             
2019-09-07  0.0   0.0  0.0  0.0


In [35]:
data.drop(data[data['광진구'] == 0].index, inplace=True)

In [36]:
data.asfreq(freq='d', method='ffill')

,광진구,동대문구,성동구,중랑구
일시,,,,
2018-01-01,0.592,0.368,0.580,0.162
2018-01-02,0.840,0.614,1.034,0.260
2018-01-03,0.828,0.576,0.952,0.288
2018-01-04,0.792,0.542,0.914,0.292
2018-01-05,0.818,0.602,0.994,0.308
...,...,...,...,...
2021-12-27,3.830,3.416,2.908,2.350
2021-12-28,4.510,3.890,3.714,2.700
2021-12-29,4.490,3.524,3.660,2.524


In [37]:
print(data[data['광진구'] <= 0])
print(data[data['성동구'] <= 0])
print(data[data['동대문구'] <= 0])
print(data[data['중랑구'] <= 0])

Empty DataFrame
Columns: [광진구, 동대문구, 성동구, 중랑구]
Index: []
Empty DataFrame
Columns: [광진구, 동대문구, 성동구, 중랑구]
Index: []
Empty DataFrame
Columns: [광진구, 동대문구, 성동구, 중랑구]
Index: []
Empty DataFrame
Columns: [광진구, 동대문구, 성동구, 중랑구]
Index: []


In [38]:
import statsmodels.api as sm
result = sm.tsa.seasonal_decompose(x = data["광진구"],model='additive', period = 360)

In [39]:
detrended = data["광진구"] - result.trend
detrended

일시
2018-01-01   NaN
2018-01-02   NaN
2018-01-03   NaN
2018-01-04   NaN
2018-01-05   NaN
              ..
2021-12-27   NaN
2021-12-28   NaN
2021-12-29   NaN
2021-12-30   NaN
2021-12-31   NaN
Length: 1460, dtype: float64

In [40]:
df_detrended = pd.concat([data, detrended], axis=1)
#df_detrended.columns = ['original_광진구', 'detrended_광진구']

# 결과를 출력합니다.
print(df_detrended.head())

              광진구   동대문구    성동구    중랑구   0
일시                                        
2018-01-01  0.592  0.368  0.580  0.162 NaN
2018-01-02  0.840  0.614  1.034  0.260 NaN
2018-01-03  0.828  0.576  0.952  0.288 NaN
2018-01-04  0.792  0.542  0.914  0.292 NaN
2018-01-05  0.818  0.602  0.994  0.308 NaN
